In [1]:
import ee


In [2]:

#ee.Authenticate()
ee.Initialize()
import geemap
Map = geemap.Map(center=[36,-120], zoom =8)


Make stack of LANDFIRE inputs:
    BPS code
    BPS name
    BPS model
    EVH
    EVC
    Lat/long

In [6]:
#AOI
aoi = ee.Geometry.BBox(-121.351882, 36.480156 , -120.861284,36.874108)
#Start geemap 
Map = geemap.Map(center=[36,-120], zoom =8)


bps = ee.ImageCollection("LANDFIRE/Vegetation/BPS/v1_4_0").reduce(ee.Reducer.max())
#rapcovers = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").reduce(ee.Reducer.max())
evh = ee.ImageCollection("LANDFIRE/Vegetation/EVH/v1_4_0").reduce(ee.Reducer.max())
rapcoverha = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('AFG')
rapcoverhp = ee.Image("projects/rap-data-365417/assets/vegetation-cover-v3/2014").select('PFG') 
sclass = ee.ImageCollection("LANDFIRE/Fire/SClass/v1_4_0").reduce(ee.Reducer.max())

rapcoverh = rapcoverha.add(rapcoverhp)

col = ee.ImageCollection([])
for year in range(2019,2024):
    # Define the GeoTIFF file path in the GCS bucket
    geotiff_path = f'gs://gigafire_rvs/rpms/rpms_{year}.tif' 
    
    # Load the GeoTIFF as an image
    image = ee.Image.loadGeoTIFF(geotiff_path)
    
    # Add the image to the image collection
    col = col.merge(ee.ImageCollection([image]))

# Print the image collection
print("Image Collection:", col.getInfo())


percentiles = [5, 25, 50, 75, 95]


rpms_percentiles = col.reduce(ee.Reducer.percentile(percentiles))

col = ee.ImageCollection("LANDSAT/LC08/C01/T1_32DAY_NDVI")

def annual_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_ndvi = col.filterDate(start_date, end_date).median().set('year', year)
    return annual_ndvi
    
    # Create annual composites for each year in the range
start_year = 2013
end_year = 2021
annual_ndvi_collection = ee.ImageCollection.fromImages(
    [annual_composite(year) for year in range(start_year, end_year + 1)])

# Compute percentiles
ndvi_percentiles = annual_ndvi_collection.reduce(ee.Reducer.percentile(percentiles))

#Gridmet precip
gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").select(['pr'])

#Gridmet annual sums
def annual_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_precip = gridmet.filterDate(start_date, end_date).sum().set('year', year)
    return annual_precip
    
    # Create annual composites for each year in the range
start_year = 1979
end_year = 2023
annual_precip_collection = ee.ImageCollection.fromImages(
    [annual_composite(year) for year in range(start_year, end_year + 1)])

precip_percentiles = annual_precip_collection.reduce(ee.Reducer.percentile(percentiles))


landfire_stack = ee.Image(
        [bps, rapcoverh, evh,ndvi_percentiles, precip_percentiles, sclass,rpms_percentiles]).toFloat()








Image Collection: {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'B0', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [136595, 87376], 'crs': 'GEOGCS["WGS 84", \n  DATUM["World Geodetic System 1984", \n    SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n    AUTHORITY["EPSG","6326"]], \n  PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Geodetic longitude", EAST], \n  AXIS["Geodetic latitude", NORTH], \n  AUTHORITY["EPSG","4326"]]', 'crs_transform': [0.00026949458523585647, 0, -124.84902599680046, 0, -0.00026949458523585647, 49.38461324988546]}], 'properties': {'system:index': '1_1_1_1_2_0'}}, {'type': 'Image', 'bands': [{'id': 'B0', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [136595, 87376], 'crs': 'GEOGCS["WGS 84", \n  DATUM["World Geodetic System 1984", \n 

In [7]:
from google.cloud import storage

bucket_id = "gigafire_rvs"

client = storage.Client()
bucket = client.get_bucket(bucket_id)
# list all objects in the directory
blobs = bucket.list_blobs(prefix="lf")
for blob in blobs:
    blob.delete()

#Export tiles to bucket (shard size 256)
def export_stack_to_cloud_storage(stack):
    # ee_region_bbox = ee_region.bounds()
    # print(ee_region_bbox.coordinates())
    task = ee.batch.Export.image.toCloudStorage(
        image=stack,
        fileNamePrefix="lf",
        bucket=bucket_id,
        scale=30,
        crs='EPSG:4326',
        shardSize=256,
        region=aoi,
        fileDimensions=256,
        skipEmptyTiles=True,
        maxPixels=1e13,
        fileFormat="GeoTIFF",
        formatOptions={
            'cloudOptimized': True
        }
    )
    task.start()

export_stack_to_cloud_storage(landfire_stack)


In [8]:
###Download from bucket to tmp folder

from google.cloud import storage
import os
import tempfile

bucket_id = "gigafire_rvs"
#wd = "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/temp"
# Create a temporary directory to download the files
temp_dir = tempfile.mkdtemp()
print(temp_dir)

def list_files_in_bucket(bucket_name):
    """List all files in a Google Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    files = []
    blobs = bucket.list_blobs(prefix="lf")
    for blob in blobs:
        files.append(blob.name)

    return files

def download_files_from_bucket(bucket_name, local_temp_dir):
    """Download files from a Google Cloud Storage bucket to a local directory."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    local_files = []
    blobs = bucket.list_blobs(prefix="lf")
    for blob in blobs:
        local_file_path = os.path.join(local_temp_dir, blob.name)
        blob.download_to_filename(local_file_path)
        local_files.append(local_file_path)

    return local_files




# Download files to temporary directory
local_files = download_files_from_bucket(bucket_id, temp_dir)
print("Downloaded files:", local_files)





C:\Users\ingli\AppData\Local\Temp\tmp56galbc_
Downloaded files: ['C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000000000.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000000256.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000000512.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000000768.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000001024.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000001280.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000001536.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000000-0000001792.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000256-0000000000.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000256-0000000256.tif', 'C:\\Users\\ingli\\AppData\\Local\\Temp\\tmp56galbc_\\lf0000000256-0000000512.tif', 'C:\\Users\

In [9]:
##Geotiff to dataframe

import rasterio
import pandas as pd




def multiband_geotiff_to_dataframe(geotiff_path):
    """Converts a multiband GeoTIFF file into a pandas DataFrame with x and y coordinates."""
    # Open the GeoTIFF file
    with rasterio.open(geotiff_path) as src:
        # Read all bands
        num_bands = src.count
        band_data = [src.read(i) for i in range(1, num_bands + 1)]
        
        # Get metadata
        transform = src.transform
        height, width = band_data[0].shape

        # Generate x and y coordinates
        x_coords = []
        y_coords = []
        for i in range(height):
            for j in range(width):
                x, y = src.xy(i, j)
                x_coords.append(x)
                y_coords.append(y)

        # Flatten band data and coordinates into DataFrame
        df_data = {'x': x_coords, 'y': y_coords}
        for band_idx in range(num_bands):
            band_name = src.descriptions[band_idx] if src.descriptions else f'band_{band_idx + 1}'
            df_data[band_name] = band_data[band_idx].flatten()

        df = pd.DataFrame(df_data)

    return df


dfs = [multiband_geotiff_to_dataframe(file_path) for file_path in local_files]

# Clean up temporary directory
for file_path in local_files:
    os.remove(file_path)
os.rmdir(temp_dir)

# Concatenate the list of DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

ndvi_columns = [col for col in combined_df.columns if col.startswith('ndvi_')]

# Raise values in selected columns to the power of 4
combined_df[ndvi_columns] = combined_df[ndvi_columns].apply(lambda x: x ** 4)
#Take subset 

# Display the DataFrame
print(combined_df.head())


            x          y  BPS_max   AFG  EVH_max   NDVI_p5  NDVI_p25  \
0 -121.351930  36.874001    955.0  58.0    101.0  0.218912  0.223409   
1 -121.351660  36.874001    950.0  56.0    104.0  0.218921  0.231732   
2 -121.351391  36.874001    950.0  57.0    104.0  0.218921  0.231732   
3 -121.351121  36.874001    950.0  59.0    104.0  0.226918  0.247927   
4 -121.350852  36.874001    950.0  46.0    104.0  0.219087  0.225123   

   NDVI_p50  NDVI_p75  NDVI_p95  ...      pr_p25      pr_p50      pr_p75  \
0  0.238541  0.255922  0.271181  ...  306.881958  375.137115  445.885406   
1  0.250986  0.253048  0.284056  ...  306.881958  375.137115  445.885406   
2  0.250986  0.253048  0.284056  ...  306.881958  375.137115  445.885406   
3  0.256732  0.264414  0.290317  ...  306.881958  375.137115  445.885406   
4  0.229193  0.245641  0.265546  ...  306.881958  375.137115  445.885406   

       pr_p95  SClass_max   B0_p5  B0_p25  B0_p50  B0_p75  B0_p95  
0  637.093018         7.0     NaN     NaN 

In [14]:
##Join df to kattribute table and reorder and clean columns
key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/LF_140BPS_01122015.csv")
#cover_key = pd.read_csv("C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/cover_key.csv")
join = combined_df.join(key[['VALUE','BPS_CODE', 'BPS_NAME','BPS_MODEL']].set_index('VALUE'), on='BPS_max')
#join=join.join(cover_key[['cover']].set_index('VALUE'), on='EVC_max')


join = join.assign(PLOT_ID=range(len(join)))

cols = list(join.columns)


Plots_Active = join[['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 
                     'AFG', 'EVH_max',
                     'y', 'x', *cols[10:15], *cols[5:10], 'SClass_max', *cols[16:21]]]


#Plots_Active = Plots_Active.iloc[:200]
cols = list(Plots_Active.columns)
print(cols)



['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 'AFG', 'EVH_max', 'y', 'x', 'pr_p5', 'pr_p25', 'pr_p50', 'pr_p75', 'pr_p95', 'NDVI_p5', 'NDVI_p25', 'NDVI_p50', 'NDVI_p75', 'NDVI_p95', 'SClass_max', 'B0_p5', 'B0_p25', 'B0_p50', 'B0_p75', 'B0_p95']


In [16]:
import sqlite3
import pandas as pd
import os
import shutil

# Copy template to home folder for reading and writing 
source_file = 'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/test/rvs_demo_fromRobbAddFieldTypeMissing.db'
destination_file = 'C:/Users/ingli/Desktop/home/240321_landfire_plots_cover.db'
shutil.copy(source_file, destination_file)
print("File copied successfully.")



# Path to the SQLite database file
database_path = 'C:/Users/ingli/Desktop/home/240321_landfire_plots_cover.db'
print(database_path)

# Create a connection to the database
conn = sqlite3.connect(database_path)

# Define the table name
table_name = "Plots_Active"

# Get the column names of the table
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({table_name})")
cols= [column[1] for column in cursor.fetchall()]  + ['NPP_1', 'NPP_2', 'NPP_3', 'NPP_4', 'NPP_5']
print(cols)
# Read data from pandas DataFrame
# Assuming df is your pandas DataFrame with the same column names


Plots_Active.columns=cols
# Replace values in the table with pandas DataFrame values
Plots_Active.to_sql(table_name, conn, if_exists='replace', index=False)

#Delete herb_cover and herb_height
columns_to_update = ['herb_height']

# Execute SQL UPDATE statements to remove values from specified columns
for column in columns_to_update:
    conn.execute(f"UPDATE {table_name} SET {column} = NULL")  # Set values to NULL
    #conn.execute(f"ALTER TABLE {table_name} DROP COLUMN {column}")


table_name="Shrubs_Active"
cursor.execute(f"DELETE FROM {table_name}")

table_name="Disturbance_Plots"
cursor.execute(f"DROP TABLE {table_name}")

table_name="Disturbance_Plots_NoFire"
cursor.execute(f"DROP TABLE {table_name}")

table_name="Dist_year7fire"
cursor.execute(f"DROP TABLE {table_name}")

#conn.execute(f"UPDATE Plots_Active SET sclass = 10") 

# Commit changes and close connection
conn.commit()
conn.close()

#Copy file from home to RVS git folder for RVS use
source_file = 'C:/Users/ingli/Desktop/home/240321_landfire_plots_cover.db'
destination_file = 'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/240321_landfire_plots_cover.db'
shutil.copy(source_file, destination_file)
print("File copied successfully.")


File copied successfully.
C:/Users/ingli/Desktop/home/240321_landfire_plots_cover.db
['PLOT_ID', 'BPS_CODE', 'BPS_NAME', 'BPS_MODEL', 'herb_cover', 'herb_height', 'latitude', 'longitude', 'PPT_1', 'PPT_2', 'PPT_3', 'PPT_4', 'PPT_5', 'NDVI_1', 'NDVI_2', 'NDVI_3', 'NDVI_4', 'NDVI_5', 'sclass', 'NPP_1', 'NPP_2', 'NPP_3', 'NPP_4', 'NPP_5']
File copied successfully.


In [82]:
rows = [200]

# DataFrame to store execution times
execution_times_df = pd.DataFrame(columns=['Rows', 'Execution Time (seconds)'])

import time
import pandas as pd

print(len(combined_df))
# Number of iterations

# DataFrame to store execution times
execution_times_df = pd.DataFrame(columns=['Rows', 'Execution Time (seconds)'])

for i in rows:
    
    source_file = 'C:/Users/ingli/Desktop/home/240321_landfire_plots_cover.db'
    destination_file = f'C:/Users/ingli/Desktop/home/240321_landfire_plots_cover_{i}.db'
    shutil.copy(source_file, destination_file)
    print("File copied successfully.")

    conn = sqlite3.connect(destination_file)
    cursor = conn.cursor()
    

    cursor.execute(f'''
    DELETE FROM Plots_Active
    WHERE PLOT_ID NOT IN (
        SELECT PLOT_ID FROM Plots_Active
        ORDER BY PLOT_ID ASC
        LIMIT {i}
    )
    ''')
        
    conn.commit()
    conn.execute('VACUUM')
    # Close the database connection
    conn.close()


    source_file = f'C:/Users/ingli/Desktop/home/240321_landfire_plots_cover_{i}.db'
    destination_file = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/240321_landfire_plots_cover_{i}.db'
    shutil.copy(source_file, destination_file)
    print("File copied successfully.")

    source_file = f'/data/240321_landfire_plots_cover_{i}.db'

# Command with the file path containing 'i'
    command = f"docker run -v \"C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test\":\"/data\" rvs-image-2024 rvs {source_file} /data/out.db 100"
    print(command)

    start_time = time.time()
    # Execute the command
    #!{command}
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(execution_time)
    # Append execution time to DataFrame
    execution_times_df = pd.concat([execution_times_df, pd.DataFrame([{'Rows': i, 'Execution Time (seconds)': execution_time}])], ignore_index=True)

print(execution_times_df)

2663764
File copied successfully.
File copied successfully.
docker run -v "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test":"/data" rvs-image-2024 rvs /data/240321_landfire_plots_cover_200.db /data/out.db 100
0.0
  Rows  Execution Time (seconds)
0  200                       0.0


In [ ]:
print(execution_times_df)
#execution_times_df.set_index('Rows', inplace=True)
#execution_times_df.reset_index(inplace=True)
execution_times_df['Rows'] = pd.to_numeric(execution_times_df['Rows'])
execution_times_df.plot(x='Rows', y='Execution Time (seconds)', kind='line', title='RVS execution time vs. db size', xlabel='No. of rows', ylabel='Time (s)')

In [109]:
!docker run -v "C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test":"/data" rvs-image-2024-dev rvs /data/240321_landfire_plots_cover_200.db /data/out.db 100 true

rvs
/data/240321_landfire_plots_cover_200.db
/data/out.db
100
true
3.37.2
2022-01-06 13:25:41 872ba256cbf61d9290b571c0e6d82a20c224ca3ad82971edc46b29818d5dalt1
Loading records...
Done.

YEAR 0


YEAR 1


YEAR 2


YEAR 3


YEAR 4


YEAR 5


YEAR 6


YEAR 7


YEAR 8


YEAR 9


YEAR 10


YEAR 11


YEAR 12


YEAR 13


YEAR 14


YEAR 15


YEAR 16


YEAR 17


YEAR 18


YEAR 19


YEAR 20


YEAR 21


YEAR 22


YEAR 23


YEAR 24


YEAR 25


YEAR 26


YEAR 27


YEAR 28


YEAR 29


YEAR 30


YEAR 31


YEAR 32


YEAR 33


YEAR 34


YEAR 35


YEAR 36


YEAR 37


YEAR 38


YEAR 39


YEAR 40


YEAR 41


YEAR 42


YEAR 43


YEAR 44


YEAR 45


YEAR 46


YEAR 47


YEAR 48


YEAR 49


YEAR 50


YEAR 51


YEAR 52


YEAR 53


YEAR 54


YEAR 55


YEAR 56


YEAR 57


YEAR 58


YEAR 59


YEAR 60


YEAR 61


YEAR 62


YEAR 63


YEAR 64


YEAR 65


YEAR 66


YEAR 67


YEAR 68


YEAR 69


YEAR 70


YEAR 71


YEAR 72


YEAR 73


YEAR 74


YEAR 75


YEAR 76


YEAR 77


YEAR 78


YEAR 79


YEAR 80


YEAR 81


YEAR 